# AutoGEN 第一课： 通过2个agent实现代码生成、执行与调试

在本例子中，我们会通过构建2个最基础但是最核心的agent实例，带你了解一下AutoGEN的基本原理。
1. AssistantAgent: 可以对提问作出反馈、给出代码，类似于人类向gpt提问会得到结果。但是它本身并不具备执行代码的能力。
2. UserProxyAgent: 可以从人类那里得到指定、或者按照自己的理解去做一些事情。比如它可以从AssistantAgent拿到代码后，去执行。

## 1. 配置环境

1. 你需要安装autogent的库， 要求python版本>=3.8
2. 配置openai 或者 azure openai的api key, 并将key写入到一个名为 "OAI_CONFIG_LIST"的文件中，并将该文件放在项目目录下。或者将其配置到你的环境变量中。

OAI_CONFIG_LIST的内容结构如下：
```python
[
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]

In [3]:
%pip install pyautogen~=0.1.0 -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

# 构建一个assistant agent实例，并且名为 assistant
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,  # 用于复现结果的种子
        "config_list": config_list,  
        "temperature": 0,  
    },  
)

# 构建一个user proxy agent, 并命名为user_proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    #  不同的人类介入程度：
    #   never: 人类始终不介入， 
    #   always: 每一步都需要人类介入， 
    #   terminate：只有当assistant agent认为当前交互结果没问题了，会返回一个"terminate"字符串, （你可以理解为暗号）。此时user proxy agent 才会停下等人类的下一步指令。
    human_input_mode="ALWAYS",  
    max_consecutive_auto_reply=10,      # 最大的交互轮数，如果超过了会强制退出，等人类指令。
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), # 如果别的agent返回的结果中带有这个"TERMINATE"标记，也会强制退出，等人类指令。
    code_execution_config={
        "work_dir": "coding",   # 临时代码存放的目录，为coding
        "use_docker": False,    # 设定是否使用docker
    },
)

# 启动两个agent会话
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

First, let's get the current date. We can do this using Python's built-in datetime module. Here is the Python code to get the current date:

```python
import datetime

# Get the current date
current_date = datetime.date.today()

print("Today's date is:", current_date)
```

Next, we need to get the year-to-date (YTD) gain for META (Facebook) and TESLA. We can do this by using a financial data API such as Yahoo Finance or Alpha Vantage. However, these APIs require an API key and the process can be complex for a beginner. 

An easier way is to use the yfinance package in Python, which allows you to download stock data from Yahoo Finance. Here is the Python code to get the YTD gain for META and TESLA:

```python
import yfinance as yf

# Get the data for the stock
meta = yf.Ticker("META